## Fine tuning Diffusion Model v2.3b
colab実装：kizamimi, tonimono

　

### このコードで使用したプログラムの参考サイト

DiffusersベースでStable Diffusionをfine tuningする（Kohya S.様）2022/12/04

https://note.com/kohya_ss/n/nbf7ce8d80f29

In [ ]:
#@title Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15360 MiB, 15109 MiB


## Part0 データセットフォルダの作成

In [ ]:
#@title Part0-1 google driveと接続する

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Part0-2 新しいデータセットを作成する
!mkdir /content/drive/MyDrive/My_Finetuing
import os
path = "/content/drive/MyDrive/My_Finetuing"
files = os.listdir(path=path)
max_i = 0
for f in files:
  if os.path.isdir(os.path.join(path, f)):
    if(max_i < int(f)):
      max_i = int(f)
max_i += 1
!mkdir /content/drive/MyDrive/My_Finetuing/{max_i}
!mkdir /content/drive/MyDrive/My_Finetuing/{max_i}/dataset
!mkdir /content/drive/MyDrive/My_Finetuing/{max_i}/My_Custom_ckpt
print("MyDrive/My_Finetuing/"+str(max_i)+"/dataset にデータセットを入れてください")
print("MyDrive/My_Finetuing/"+str(max_i)+"/My_Custom_ckpt にカスタムモデルを入れてください")

MyDrive/My_Finetuing/1/dataset にデータセットを入れてください
MyDrive/My_Finetuing/1/My_Custom_ckpt にカスタムモデルを入れてください


## Part1 diffusion modelの追加学習

In [ ]:
#@title Part1-1 google driveと接続する

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Part1-2 Part1のどれかを使用するとき毎回実行する初期設定

%pip install diffusers[torch]==0.9.0
%pip install -q -U --pre triton
%pip install -q accelerate transformers>=4.21.0 ftfy bitsandbytes einops pytorch_lightning tensorflow-io>=0.22.0 fairscale==0.4.4 timm==0.4.12
#%pip install https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+1515f77.d20221130-cp38-cp38-linux_x86_64.whl
!pip install https://github.com/brian6091/xformers-wheels/releases/download/0.0.15.dev0%2B4c06c79/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl
!git clone https://github.com/mili-inch/finetuning_by_kohya_s.git train

#@markdown ーーーーーーーーーー訓練済みモデル(ckptのURL)の指定ーーーーーーーーー
pretrained_model_tag= "WD1.3_lite" #@param ["WD1.3_lite", "WD1.3", "SD1.4", "SD1.5", "trinart_V2_60Ksteps", "trinart_V2_95Ksteps", "trinart_V2_115Ksteps", "derrida", "trinart_characters"] {allow-input: false}
#@markdown ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

import os
import glob
path = "/content/drive/MyDrive/My_Finetuing"
files = os.listdir(path=path)
max_i = 0
for f in files:
  if os.path.isdir(os.path.join(path, f)):
    if(max_i < int(f)):
      max_i = int(f)

if(pretrained_model_tag == "custom model"):
  pretrained_model = glob.glob("/content/drive/MyDrive/My_Finetuing/"+str(max_i)+"/My_Custom_ckpt/*")[0]
elif(pretrained_model_tag == "WD1.3_lite"):
  pretrained_model = "https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float32.ckpt"
elif(pretrained_model_tag == "WD1.3"):
  pretrained_model = "https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-full.ckpt"
elif(pretrained_model_tag == "WD1.3_full"):
  pretrained_model = "https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-full-opt.ckpt"
elif(pretrained_model_tag == "SD1.4"):
  pretrained_model = "https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4-full-ema.ckpt"
elif(pretrained_model_tag == "SD1.5"):
  pretrained_model = "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt"
elif(pretrained_model_tag == "trinart_V2_115Ksteps"):
  pretrained_model = "https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step115000.ckpt"
elif(pretrained_model_tag == "trinart_V2_95Ksteps"):
  pretrained_model = "https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step95000.ckpt"
elif(pretrained_model_tag == "trinart_V2_60Ksteps"):
  pretrained_model = "https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step60000.ckpt"
elif(pretrained_model_tag == "derrida"):
  pretrained_model = "https://huggingface.co/naclbit/trinart_derrida_characters_v2_stable_diffusion/resolve/main/derrida_final.ckpt"
elif(pretrained_model_tag == "trinart_characters"):
  pretrained_model = "https://huggingface.co/naclbit/trinart_characters_19.2m_stable_diffusion_v1/resolve/main/trinart_characters_it4_v1.ckpt"

#@markdown ーーーーーーーーーーーーーーー訓練ステップ数ーーーーーーーーーーーー
max_train_steps = 1 #@param {type : "number"}
#@markdown ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

#@markdown ーーーーーーーーーーーーーーーバッチサイズーーーーーーーーーーーーー
batch_size = 4 #@param {type : "number"}
#@markdown ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

#@markdown ーーーーーーーーーーーーーーーー学習係数ーーーーーーーーーーーーーー
learning_rate = 5e-6 #@param {type : "number"}
#@markdown ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

In [ ]:
#@title Part1-3 データセットの作成処理
#a@markdown データセットのフォルダの中に追加したいプロンプトの名を持つフォルダを

import os
import glob
path = "/content/drive/MyDrive/My_Finetuing"
root_path = os.path.join(path,str(max_i))
dataset_path = os.path.join(path,str(max_i),"dataset")
files = os.listdir(path=path)
max_i = 0
for f in files:
  if os.path.isdir(os.path.join(path, f)):
    if(max_i < int(f)):
      max_i = int(f)

image_list = sorted(glob.glob(os.path.join(dataset_path,"*")))
for image in image_list:
  print(image)
  if(image[-3:] == "txt" or image[-3:] == "npz"):
      os.remove(image)

#a@markdown 書いて画像を入れるとその画像用ののプロンプト(タグ)に追加される
!git clone https://github.com/KichangKim/DeepDanbooru.git
!git -C "./DeepDanbooru" checkout c48689a85dde0e4a852c1691a7d746abe242e283
!wget https://github.com/KichangKim/DeepDanbooru/releases/download/v3-20211112-sgd-e28/deepdanbooru-v3-20211112-sgd-e28.zip
import os
import shutil
if(os.path.isdir("./deepdanbooru-model")):
  shutil.rmtree("./deepdanbooru-model")
!unzip "deepdanbooru-v3-20211112-sgd-e28.zip" -d "./deepdanbooru-model"
%pip install ./DeepDanbooru

import glob
import os
from PIL import Image
import json
import shutil

dir = os.listdir(path=os.path.join(dataset_path))

image_list = sorted(glob.glob(os.path.join(dataset_path,"*")))
for image in image_list:
  print(image)
  if(image[-3:] == "jpg"):
      im = Image.open(image)
      im.save(image[:-3]+'png')
      os.remove(image)
  if(image[-4:] == "jpeg" or image[-4:] == "jfif" or image[-4:] == "webp"):
      im = Image.open(image)
      im.save(image[:-4]+'png')
      os.remove(image)
  if(image[-3:] == "gif"):
      os.remove(image)

##@markdown ーーーーーーーーーーーーー追加するプロンプトーーーーーーーーーーーー
#add_prompt = "" #@param {type : "string"}
##@markdown ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

def make_tags(path, fil, tag, id):
  files = os.listdir(path)
  for f in files:
      path_add = os.path.join(path, f)
      if os.path.isfile(path_add):
          dic = {}
          # ファイルの場合
          dic["base_path"] = path_add
          dic["rename_path"] = os.path.join(path, tag+"_"+str(id).zfill(10)+".png")
          id += 1
          json.dump(dic, fil, ensure_ascii=False)
          fil.write('\n')
  for f in files:
      path_add = os.path.join(path, f)
      if os.path.isdir(path_add):
          #フォルダの場合
          new_tag = "_"+f
          id = make_tags(path_add, fil, tag + new_tag, id)
  return id

with open(root_path+"/dataset_add_tag.jsonl","w",encoding='utf-8') as fil:
  make_tags(dataset_path, fil, "", 0)

#TODO　ファイル構成をルートに戻す
with open(root_path+"/dataset_add_tag.jsonl","r",encoding='utf-8') as fil:
  for dic in fil:
    json_dict = json.loads(dic)
    os.rename(json_dict["base_path"],json_dict["rename_path"])
    if("/".join(json_dict["rename_path"].split("/")[:-1]) != os.path.join(dataset_path)):
      shutil.move(json_dict["rename_path"], os.path.join(dataset_path))

!deepdanbooru evaluate {dataset_path} --project-path deepdanbooru-model --allow-folder --save-txt --allow-gpu

#dataset_add_tag.txtに基づいてプロンプト情報の変更を加える

txt = sorted(glob.glob(os.path.join(dataset_path,"*.txt")))
for one in txt:
  prompt = ""
  with open(one,"r") as f:
    prompt = f.readline()
  script_prompt = "_".join(one.split("/")[-1].split("_"))
  print("script_prompt:"+str(script_prompt))
  #TODO:script_promptに基づいてプロンプトの状態を決める
  sp_prompt = prompt.split(",")
  if(len(sp_prompt) >= 4):
    prompt = ",".join([sp_prompt[0]]+[sp_prompt[-2]]+sp_prompt[1:-2]+[sp_prompt[-1]])
  if(script_prompt.split("_")[1] == "all"):
    prompt = script_prompt.split("_")[2]
  elif(script_prompt.split("_")[1] == "add"):
    max_i = max(0, int(script_prompt.split("_")[2]))
    min_i = min(max_i, len(prompt.split(",")))
    sp_prompt = prompt.split(",")
    new_prompt = script_prompt.split("_")[3]
    sp_prompt.insert(min_i, new_prompt)
    prompt = ",".join(sp_prompt)
  #prompt = add_prompt + ", " + prompt
  with open(one,"w") as f:
    f.write(prompt)

In [ ]:
#@title Part1-4 diffusion modelの追加学習

import os
path = "/content/drive/MyDrive/My_Finetuing"
dataset_path = os.path.join(path,str(max_i),"dataset")
result_path = os.path.join(path,str(max_i),"result")

!python ./train/merge_dd_tags_to_metadata.py {dataset_path} meta_cap_dd.json
!python ./train/clean_captions_and_tags.py {dataset_path} meta_cap_dd.json meta_clean.json
!wget {pretrained_model}
import os
try:
  os.rename(pretrained_model.split("/")[-1], 'base.ckpt')
except:
  pass
#--output_dir=./drive/MyDrive/{base_dir}/{result_dir}
!python ./train/prepare_buckets_latents.py {dataset_path} meta_clean.json meta_lat.json ./base.ckpt --batch_size 1 --max_resolution 512,512 --mixed_precision no
!accelerate launch --config_file=./train/default_config.yaml --num_cpu_threads_per_process 8 ./train/fine_tune.py \
    --pretrained_model_name_or_path=./base.ckpt \
    --in_json=./meta_lat.json \
    --train_data_dir={dataset_path} \
    --output_dir={result_path} \
    --shuffle_caption \
    --train_batch_size={batch_size} \
    --learning_rate=5e-6 \
    --max_train_steps={max_train_steps} \
    --use_8bit_adam \
    --xformers \
    --gradient_checkpointing \
    --mixed_precision=fp16

## Part2 ckpt形式のモデルをdiffusersに変換

In [ ]:
#@title Part2-1 google driveと接続する

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Part2-2 ckpt to diffusers
#@markdown Part2-1の実行手順

#@markdown １：一回目実行

#@markdown ２：Restart Runtimeボタンが実行画面に表示されるのですぐにクリックしてください

#@markdown ３：２でのボタンクリックで自動的に再起動

#@markdown ４：二回目実行

import os
path = "/content/drive/MyDrive/My_Finetuing"
files = os.listdir(path=path)
max_i = 0
for f in files:
  if os.path.isdir(os.path.join(path, f)):
    if(max_i < int(f)):
      max_i = int(f)
dataset_path = os.path.join(path,str(max_i),"result","last.ckpt")
diff_path = os.path.join(path,str(max_i),"diffusers_model")

%cd /content
!pip install OmegaConf
%pip install --upgrade diffusers
%pip install -q transformers>=4.21.0
!pip install textfile
import textfile
!wget https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml
!git clone https://github.com/huggingface/diffusers
!mv v1-inference.yaml ./diffusers/scripts/
%cd ./diffusers/scripts
path = "./convert_original_stable_diffusion_to_diffusers.py"
base = "HeunDiscreteScheduler,"
result = ""
textfile.replace(path,base,result)
base2 = "from diffusers.pipelines.paint_by_example import PaintByExampleImageEncoder, PaintByExamplePipeline"
result2 = "#from diffusers.pipelines.paint_by_example import PaintByExampleImageEncoder, PaintByExamplePipeline"
textfile.replace(path,base2,result2)
!python ./convert_original_stable_diffusion_to_diffusers.py --checkpoint_path {dataset_path} --dump_path {diff_path}

## Part3 追加学習したモデルで画像生成

In [ ]:
#@title Part3-1 google driveと接続する

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Part3-2 訓練済みモデルのロード
#@markdown Part3-2の実行手順

%cd /content

import os
path = "/content/drive/MyDrive/My_Finetuing"
files = os.listdir(path=path)
max_i = 0
for f in files:
  if os.path.isdir(os.path.join(path, f)):
    if(max_i < int(f)):
      max_i = int(f)
diff_path = os.path.join(path,str(max_i),"diffusers_model")

#@markdown １：実行

#@markdown ２：Part3-3に進む
!pip install --upgrade diffusers transformers scipy 
!pip install accelerate ftfy

import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained(diff_path, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
i = 0

In [ ]:
#@title Part3-3 訓練済みモデルで生成
#@markdown Part3-3の実行手順

#@markdown １：画像生成に使用するプロンプトと設定を指定してください

#@markdown ーーーーーーーーーーーーーーープロンプトーーーーーーーーーーーーーー
prompt = "1girl" #@param{type:"string"}
negative_prompt = "" #@param{type:"string"}
#@markdown ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

#@markdown ーーーーーーーーー任意の番号（ファイル名に使用します）ーーーーーーー
id = 1 #@param{type:"number"}
#@markdown ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

#@markdown ーーーーーーーーーーーーーーーランダムシードーーーーーーーーーーーー
seed = -1 #@param{type:"number"}
#@markdown ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

#@markdown ２：実行

import random
import numpy as np
import torch

exe_name = str(id).zfill(3) + "_result_"

def torch_fix_seed(seed=42):
    # Python random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True

if(seed >= 0):
  torch_fix_seed(seed=seed)


from IPython.display import Image,display_png

for j in range(5):
  if(prompt == ""):
    prompt = f.readline()
    f.close()
  pipe.enable_attention_slicing()
  image = pipe(prompt,negative_prompt=negative_prompt).images[0]  
  image.save(exe_name+"_"+str(i).zfill(4)+".png")
  display_png(Image(exe_name+"_"+str(i).zfill(4)+".png"))
  from google.colab import files
  files.download(exe_name+"_"+str(i).zfill(4)+".png")
  i+=1